In [1]:
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import QtWidgets
from PyQt5 import QtGui
from PyQt5 import QtCore, QtGui, QtWidgets
from collections import deque

import os
import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import copy
import cv2
import imutils
from collections import Counter
counter = 0
buffer = 32
pts = deque(maxlen=buffer)
D = []
(dX,dY) = (0,0)
direction = ""
center ="None"

# Disable tensorflow compilation warnings
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import tensorflow as tf

def Track (hsv, frame):
    
    
    lower= np.array([0,48,80], dtype="uint8")
    upper = np.array([20,255,255], dtype="uint8")
# initialize the list of tracked point, the frame counter.the cordinates
    
    
    
    

    mask = cv2.inRange(hsv, lower,upper)
    mask = cv2.erode(mask, None, iterations = 2)
    mask = cv2.dilate(mask,None,iterations = 2)
    
    
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = "None"
    
    if len(cnts) > 0:
            
            c= max(cnts, key=cv2.contourArea)
            ((x,y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
            
            
            if radius > 10:
                cv2.circle(frame, (int(x), int(y)), int(radius), (0,255,255),2 )
                cv2.circle(frame, center, 5, (0,0,255), -1)
                pts.appendleft(center)
                
            
            
            for i in np.arange(1,len(pts)):
                if pts[i - 1] is None or pts[i] is None:
                    
                    continue
                   
                global counter
                global direction
                global dX,dY,D
                
                if counter >= 10 and i == 10  and pts[i-10] is not None :
                    
                    
                    dX = pts[i-10][0] - pts [i][0]
                    dY = pts[i-10][1] - pts [i][1]
                    (dirX, dirY) = ("","")
                    
    
                    
                    if np.abs(dX)> 10 :
                        
                        dirX= "East" if np.sign(dX) == 1 else "West"
                        D.append(dirX)
                    if np.abs(dY) > 10 :
                        dirY= "North" if np.sign(dY) == 1 else "South"
                        D.append(dirY)
                    if dirX !="" and dirY !="":
                        direction = "{}-{}".format(dirY,dirX)
                        D.append(direction)
                    else:
                        direction  = dirX if dirX !="" else dirY
                        D.append(direction)
                
                thicknees = 2
                counter +=1  
                cv2.line(frame, pts[i-1], pts[i], (0,0,255), thicknees)
                
                
                 
            
def predict(image_data):

    predictions = sess.run(softmax_tensor, \
             {'DecodeJpeg/contents:0': image_data})

    # Sort to show labels of first prediction in order of confidence
    top_k = predictions[0].argsort()[-len(predictions[0]):][::-1]

    max_score = 0.0
    res = ''
    for node_id in top_k:
        human_string = label_lines[node_id]
        score = predictions[0][node_id]
        if score > max_score:
            max_score = score
            res = human_string
    return res, max_score

# Loads label file, strips off carriage return
label_lines = [line.rstrip() for line
                   in tf.gfile.GFile("logs/trained_labels.txt")]

# Unpersists graph from file
with tf.gfile.FastGFile("logs/trained_graph.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

    
x = None
text = ""
norec = False
with tf.Session() as sess:  
    
    def getVowel(x,direction):
        amharic_letters = [["ሀ","ሁ","ሂ","ሃ","ሄ","ህ","ሆ"],["ለ","ሉ","ሊ","ላ","ሌ","ል","ሎ"],["ሐ","ሑ","ሒ","ሓ","ሔ","ሕ","ሖ"],["መ","ሙ","ሚ","ማ","ሜ","ም","ሞ"]]
        if (direction=="East"):
            return amharic_letters[x-1][1]
        elif(direction=="West"):
            return amharic_letters[x-1][2]
        elif(direction=="North"):
            return amharic_letters[x-1][3]
        elif(direction=="South"):
            return amharic_letters[x-1][4]
        elif(direction=="South-West" or direction=="South-East"):
            return amharic_letters[x-1][5]
        elif(direction=="North-West" or direction=="North-East"):
            return amharic_letters[x-1][6]
        elif(direction==""):
            return amharic_letters[x-1][0]
        else:
            return ""
        
    def getLetter(x):
        amharic_letters = ["ሀ","ለ","ሐ","መ"]
        return amharic_letters[x-1]

    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')
    class Thread(QThread):
        changePixmap = pyqtSignal(QImage)
        
        def run(self):
            # Feed the image_data as input to the graph and get first prediction

            c = 0

            cap = cv2.VideoCapture(0)

            res, score = '', 0.0
            i = 0
            mem = ''
            consecutive = 0
            sequence = ''
            
            
            while True:
                ret, frame = cap.read()
                frame=cv2.flip(frame,1)
                x1,y1,x2,y2 = 150,150,350,350
                img_cropped= frame[y1:y2, x1:x2]
                global D
                global norec
                c += 1
                image_data = cv2.imencode('.jpg', img_cropped)[1].tostring()
                cv2.rectangle(frame,(x1,y1), (x2,y2), (255,0,0),2 )
                rgbImage = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                convertToQtFormat = QImage(rgbImage.data, rgbImage.shape[1], rgbImage.shape[0], QImage.Format_RGB888)
                p = convertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)
                self.changePixmap.emit(p)
                
                #tensor
                res_tmp, score = predict(image_data)
                
                #print(score)
                res = res_tmp
                i = 0
                if i == 4:
                    if mem == res:
                        consecutive += 1
                    else:
                        consecutive = 0
                    if consecutive == 2 and res not in ['nothing']:
                        if res == 'space':
                            sequence += ' '
                            print("C")
                        elif res == 'del':
                            sequence = sequence[:-1]
                            print("B")
                        else:
                            sequence += res
                            print("A")
                        consecutive = 0
                i+=1
                global text
                if (res.upper()=="NOTHING"):
                    text+=""
                    norec = False
                elif (norec):
                    continue
                elif (res.upper()=="SPACE"):
                    text+=" "
                elif (res.upper()=="DEL"):
                    text= text[0:-2]
                else:
                    for i in range(0,20):
                        (grabbed,img) = cap.read()
                        img=cv2.flip(img,1)
                        img = imutils.resize(img, width=600)
                        x1,y1,x2,y2 = 100,100,400,400
                        cv2.rectangle(img,(x1,y1), (x2,y2), (255,0,0),2 )
                        x11,y11,x22,y22 = 200,300,200,300
                        cv2.rectangle(img,(x11,y11), (x22,y22), (255,0,0),2 )
                        frame= img[y1:y2, x1:x2]
                        blurred = cv2.GaussianBlur(frame,(11,11),0)
                        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                        Track(hsv,frame)
                        cv2.putText(img, direction,(10,30), cv2.FONT_HERSHEY_SIMPLEX,0.65,(0,0,255),3)
                        cv2.putText(img, "dX: {},dY: {}".format(dX,dY), (10, frame.shape[0]-10),cv2.FONT_HERSHEY_SIMPLEX,0.35,(0,255,255),1)
                        rgbImage = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        convertToQtFormat = QImage(rgbImage.data, rgbImage.shape[1], rgbImage.shape[0], QImage.Format_RGB888)
                        p = convertToQtFormat.scaled(640, 480, Qt.KeepAspectRatio)
                        self.changePixmap.emit(p)
                        
                    norec = True
                    
                    if D == []:
                        text += getLetter(int(res.upper()))
                        
                    else:
                        value  = Counter(D).most_common(1)[0][0]
                        print(value)
                        text += getVowel(int(res.upper()), value)
                
                D =[]
                global x
                pts.clear()
                x.setText(text)
                
                

    class App(QWidget):
        def __init__(self):
            super().__init__()
            self.title = 'Eth SL'
            self.left = 100
            self.top = 100
            self.width = 640
            self.height = 480

            self.initUI()

        @pyqtSlot(QImage)
        def setImage(self, image):
            self.label.setPixmap(QPixmap.fromImage(image))
        def initUI(self):
            self.setWindowTitle(self.title)
            self.setWindowIcon(QtGui.QIcon('logo.png'))
            self.setGeometry(self.left, self.top, self.width, self.height)
            self.resize(1000, 550)
            # create a label
            self.hsv_title = QLabel(self)
            self.hsv_title.move(650, 10)
            font = QtGui.QFont("Roboto", 32, QtGui.QFont.Bold)
            self.hsv_title.setFont(font)
            self.hsvti
            self.label.resize(640, 480)
            self.le = QLineEdit(self)
            self.le.move(0, 480)
            #font = self.le.font()      # lineedit current font
            #font.setPointSize(32)  # change it's size
            palette = QtGui.QPalette()
            palette.setColor(QtGui.QPalette.Text, QtCore.Qt.red)
            self.le.setPalette(palette)
            font = QtGui.QFont("Noto Sans Ethiopic", 32, QtGui.QFont.Bold)
            self.le.setFont(font)      # set font
            self.le.resize(640,70)
            global x
            x = self.le
            #self.le.setText(getVowel(1,"east"))
            self.show()
            th = Thread(self)
            th.changePixmap.connect(self.setImage)
            th.start()
    if __name__ == '__main__':
        app = QtWidgets.QApplication(sys.argv)
        app.setStyle('Fusion')
        palette = QtGui.QPalette()
        palette.setColor(QtGui.QPalette.Window, QtGui.QColor(53,53,53))
        palette.setColor(QtGui.QPalette.WindowText, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.Base, QtGui.QColor(15,15,15))
        palette.setColor(QtGui.QPalette.AlternateBase, QtGui.QColor(53,53,53))
        palette.setColor(QtGui.QPalette.ToolTipBase, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.ToolTipText, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.Text, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.Button, QtGui.QColor(53,53,53))
        palette.setColor(QtGui.QPalette.ButtonText, QtCore.Qt.white)
        palette.setColor(QtGui.QPalette.BrightText, QtCore.Qt.red)

        palette.setColor(QtGui.QPalette.Highlight, QtGui.QColor(0,116,255).lighter())
        palette.setColor(QtGui.QPalette.HighlightedText, QtCore.Qt.black)
        app.setPalette(palette)
        ex = App()
        exit(app.exec())

South
East
North
